**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Bruce Wayne: bruce.w@wayne-entreprises.com
*   Alfred Pennyworth: alfred@msn.com
*   Richard Grayson: robin_132@gmail.com
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #3

El propósito de este último trabajo práctico es poder hacer una aproximación inicial a los conceptos del aprendizaje supervisado.

El objetivo no es alcanzar el modelo definitivo con sus parámetros óptimos, sino más bien adquirir práctica en la ejecución de los pasos esenciales dentro de un proceso de aprendizaje supervisado.

Desde comprender la pregunta de investigación hasta la preparación de los datos, desde la segmentación del conjunto de datos hasta la evaluación del modelo. Para llevar a cabo esta actividad, emplearemos los conjuntos de datos generados en el TP2

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Limpieza de Datos y Variables
A pesar que mucho del trabajo de preparación de los datos ya se realizó en los TPs anteriores, aún falta preparar los datos para poder alimentar un modelo de clasificación. Aplicar técnicas de transformación, encoding y escalado (si hace falta).

## Ejercicio 2: Testing de Modelos

Para modelar una clasificación multi-clase existen diversas opciones. Explorar modelos de clasificación binaria aplicando técnicas OvR, OvO, etc. Elegir 3 modelos diferentes y compararlos.

Explorar también otros modelos que tengan múltiples salidas como decision trees, y sus técnicas derivadas (como random forest por ejemplo).

La idea es hacer un primer "barrido" con diferentes modelos con todos los hiperparámetros con sus valores por default, que permita evaluar qué algoritmo se ajusta mejor al problema, para luego pasar (en el siguiente ejercicio) a una optimización del mejor modelo ajustando los hiperparámetros.

## Ejercicio 3: Optimización de Hiperparámetros

Evaluar las distintas métricas (precision, recall, f1-score y support) para visualizar la performance del modelo. Armar la matriz de confusión y contrastar los resultados del modelo ya optimizado con los primeros resultados obtenidos con los primeros modelos testeados.

## Ejercicio 4: Conclusión
Con todo el recorrido realizado, elaborar una conclusión final sobre la mentoría. Dentro de la conclusión, considerar lo que se ha aprendido en las diferentes entregas (TP1 y TP2).

Parte de estas conclusiones les servirán también para el video final donde se presente el caso.

El video que sigue a este práctico serviría como informe de resultados y observaciones de este práctico, resaltando aprendizajes y problemáticas con las que se hayan encontrado.

Recuerden que un buen resultado no implica necesariamente encontrar el mejor algoritmo, sino poder asimilar un proceso de modelado completo, desde el dataset bruto hasta la optimización final.

# Desarrollo

In [ ]:
# Link al dataset: https://drive.google.com/file/d/1jo9cZNHAPpsTosW-2JA4CLo4SaFCHfIs/view?usp=drive_link

Importo las librerías necesarias:

In [1]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

pd.set_option('display.max_rows', None)

df = pd.read_csv("./dataset_concatenado.csv" , encoding='latin-1')

Me aseguro que todo está compuesto por strings

In [2]:
import pandas as pd
df['objeto'] = df['objeto'].astype(str)
df['rubro'] = df['rubro'].astype(str)
df['agencia'] = df['agencia'].astype(str)

Aplico una tokenización

In [3]:
df['rubro_tokens'] = df['rubro'].apply(lambda x: word_tokenize(x))
df['agencia_tokens'] = df['agencia'].apply(lambda x: word_tokenize(x))
df['objeto_tokens'] = df['objeto'].apply(lambda x: word_tokenize(x))

Veamos cómo va quedando:

In [4]:
df.head()

,index,referencia,objeto,rubro,agencia,apertura,monto,fuente,rubro_tokens,agencia_tokens,objeto_tokens
0,347,ContrataciÃ³n Directa 464/2017,"['bols', 'aliment']","['agric', 'ganaderia', 'caza', 'silvicult']","['com', 'nacional', 'energi', 'atom']",2017-09-29,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'com, ', ,, 'nacional, ', ,, 'energi, ', ,...","[[, 'bols, ', ,, 'aliment, ', ]]"
1,348,Subasta PÃºblica 48/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-26,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l..."
2,349,Subasta PÃºblica 47/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-24,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l..."
3,350,Subasta PÃºblica 46/2017,"['cabez', 'gan', 'vacun', 'lot', 'vac', 'inver...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-19,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l..."
4,351,Subasta PÃºblica 45/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-12,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l..."


Transformo cada token distinto en un número, y convierto todos los tokens:

In [5]:
def assign_numbers(tokens):
    
    token_to_number = {}
    number_to_token = {}
    for i, token in enumerate(tokens):
        token_to_number[token] = i
        number_to_token[i] = token
    
    return token_to_number, number_to_token

rubro_token_to_number, rubro_number_to_token = assign_numbers(df['rubro_tokens'].explode().unique())
agencia_token_to_number, agencia_number_to_token = assign_numbers(df['agencia_tokens'].explode().unique())
objeto_token_to_number, objeto_number_to_token = assign_numbers(df['objeto_tokens'].explode().unique())

df['rubro_numerical'] = df['rubro_tokens'].apply(lambda x: [rubro_token_to_number[token] for token in x])
df['agencia_numerical'] = df['agencia_tokens'].apply(lambda x: [agencia_token_to_number[token] for token in x])
df['objeto_numerical'] = df['objeto_tokens'].apply(lambda x: [objeto_token_to_number[token] for token in x])

Idea: hacer un one-hot encoding: transformamos cada columna (rubro_numerical, agencia_numerical y objeto_numerical) a una matriz de dimensión $(L,N_\text{max}N_\text{u})$, donde $N_\text{max}$ es el largo de la entrada más larga de la columna y $N_\text{u}$ es la cantidad única de tokens de nuestro diccionario. De esta forma, para una dada fila de una columna (digamos que hay una lista $X$ de tokens $x_j$), el $j-$ésimo elemento de la matriz será 1 si $x_1$ es el valor primer valor del diccionario, y 0 en el resto. Pasamos a $x_2$ y hacemos lo mismo, hasta terminar con la lista. Si el largo de $X$ es menor que $N_\text{max}$, el resto de las entradas de la matriz correspondiente es cero.

Repetimos esto para las otras columnas, concatenamos a lo largo de las filas, y obtenemos el one-hot encoding.

**Nota de Santi: CREO que el encoding de palabras a enteros tiene que estar hecho sobre los valores únicos de todo el diccionario. Si no, un 1 en rubro no tiene relación con un 1 en objeto y el sistema no va a aprender nada porque las variables no están correlacionadas, o va a aprender cualquier cosa.

In [29]:
#fdunciones auxiliares
def valores_unicos(df,col):
    #obtengo los tokens unicos de todo el dataset
    
    return df[col].explode().unique()


def longest_entry(df,col):
    #devuelve la longitud de la lista mas larga en la columna
    
    def lon_lst(lst):
        return len(lst)

    return df[col].apply(lon_lst).max()
    
#------------------------------------------------------------------------------------

#creo dataset nuevo con solo las variables numericas nuevas
df_num = df[['rubro_numerical','agencia_numerical','objeto_numerical']].copy()

#obtengo los tokens unicos
rub_unique = valores_unicos(df_num, 'rubro_numerical')
age_unique = valores_unicos(df_num, 'agencia_numerical')
obj_unique = valores_unicos(df_num, 'objeto_numerical')

#obtengo el largo de la entrada mas larga
log_en_rub = longest_entry(df_num, 'rubro_numerical')
log_en_age = longest_entry(df_num, 'agencia_numerical')
log_en_obj = longest_entry(df_num, 'objeto_numerical')

print('cantidad de columnas de rubro si hago one-hot encoding:',log_en_obj*len(df_num))
print('esto va a ocupar:',32*27549589*100000 / (8*1024**3),'GB')



cantidad de columnas de rubro si hago one-hot encoding: 27549589
esto va a ocupar: 10263.021662831306 GB


In [6]:
df.head()

,index,referencia,objeto,rubro,agencia,apertura,monto,fuente,rubro_tokens,agencia_tokens,objeto_tokens,rubro_numerical,agencia_numerical,objeto_numerical
0,347,ContrataciÃ³n Directa 464/2017,"['bols', 'aliment']","['agric', 'ganaderia', 'caza', 'silvicult']","['com', 'nacional', 'energi', 'atom']",2017-09-29,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'com, ', ,, 'nacional, ', ,, 'energi, ', ,...","[[, 'bols, ', ,, 'aliment, ', ]]","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 1, 2, 3, 4, 2, 5]"
1,348,Subasta PÃºblica 48/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-26,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l...","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 8, 2, 3, 9, 2, 3, 10, 2, 7]","[0, 6, 2, 3, 7, 2, 3, 8, 2, 3, 9, 2, 3, 10, 2,..."
2,349,Subasta PÃºblica 47/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-24,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l...","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 8, 2, 3, 9, 2, 3, 10, 2, 7]","[0, 6, 2, 3, 7, 2, 3, 8, 2, 3, 9, 2, 3, 10, 2,..."
3,350,Subasta PÃºblica 46/2017,"['cabez', 'gan', 'vacun', 'lot', 'vac', 'inver...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-19,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l...","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 8, 2, 3, 9, 2, 3, 10, 2, 7]","[0, 6, 2, 3, 7, 2, 3, 8, 2, 3, 9, 2, 3, 21, 2,..."
4,351,Subasta PÃºblica 45/2017,"['cabez', 'gan', 'vacun', 'lot', 'novill', 'go...","['agric', 'ganaderia', 'caza', 'silvicult']","['mayor', 'general', 'ejercit']",2017-10-12,0.0,Argentinacompra,"[[, 'agric, ', ,, 'ganaderia, ', ,, 'caza, ', ...","[[, 'mayor, ', ,, 'general, ', ,, 'ejercit, ', ]]","[[, 'cabez, ', ,, 'gan, ', ,, 'vacun, ', ,, 'l...","[0, 1, 2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 7]","[0, 8, 2, 3, 9, 2, 3, 10, 2, 7]","[0, 6, 2, 3, 7, 2, 3, 8, 2, 3, 9, 2, 3, 10, 2,..."


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dividir los datos en características (X) y etiquetas (y)
X = df[['agencia_encoded', 'objeto_encoded','monto']]  # Ajusta las características según tus necesidades
y = df['rubro_encoded']  # Ajusta 'clase' según la columna de etiquetas en tu dataset

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
X_train/=np.amax(X_train,axis=0)
X_test/=np.amax(X_test,axis=0)


y_train.head(100)

#y_train/=np.amax(y_train)
#y_test/=np.amax(y_test)



# # Crear y entrenar el modelo Random Forest
# rf_model = RandomForestClassifier(n_estimators=10, random_state=13)
# rf_model.fit(X_train, y_train)

# # Realizar predicciones en el conjunto de prueba
# y_pred = rf_model.predict(X_test)

# # Evaluar el rendimiento del modelo
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

# print("Accuracy:", accuracy)
# print("Classification Report:\n", report)


In [ ]:
from sklearn.tree import DecisionTreeClassifier


tree_ = DecisionTreeClassifier(max_depth=3, min_samples_leaf=3, random_state=13)
tree_.fit(X_train, y_train)
y_pred = tree_.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# Escalar las características (esto puede variar según las características)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear una arquitectura de red neuronal simple
model = keras.Sequential([
    keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='softmax')  # Ajusta el número de clases
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

print(model.get_weights())

# # Entrenar la red neuronal
# model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# # Evaluar el rendimiento del modelo en el conjunto de prueba
# test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
# print("Test Accuracy:", test_accuracy)
